In [1]:
import weaviate
from weaviate.classes.config import Property, DataType, Configure, ReferenceProperty
from sentence_transformers import SentenceTransformer
from yaml import safe_load
from sys import path
import os
from sys import path
path.append(r'C:\Users\Izogie\Desktop\Folders\Projects\Python\KB Chat\src')
from modules.SourceManager import SourceManager

c:\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Load API key from config.yaml (`GOOGLE_API_KEY: <YOUR API TOKEN>`) into environment variable.
# Kept in a seperate file for ease of access across projects.
def load_config(file_path="./config.yml"):
    with open(file_path, 'r') as file:
        config = safe_load(file)
        for key, value in config.items():
            os.environ[key] = value

load_config()

In [3]:
header = {"X-Google-Api-Key": os.environ["GOOGLE_API_KEY"],
          "X-Google-Studio-Api-Key": os.environ["GOOGLE_API_KEY"],
          "X-PaLM-Api-Key": os.environ["GOOGLE_API_KEY"],
          "X-Huggingface-Api-Key": os.environ["HF_API_KEY"],
          "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]}

In [4]:
def create_weaviate_schema(client):
    # Delete all existing classes (optional, for a fresh start)
    client.collections.delete_all()

    schema_paragraph = {
                "class": "Paragraph",
                "description": "a paragraph with a header and and parent Article",
                "vectorizer": "none",
                "moduleConfig": {
                    "text2vec-huggingface": {
                        "model": "sentence-transformers/all-MiniLM-L6-v2",
                        "options": {
                            "waitForModel": True,
                            "useGPU": True,
                            "useCache": True
                            },
                    },
                    "generative-palm":{
                        "projectId": "Generative Language Client",
                        "apiEndpoint": "generativelanguage.googleapis.com",
                        "modelId": "gemini-1.5-flash-latest",
                    },

                },
                "properties": [
                    {
                        "dataType": ["text"],
                        "description": "Title/header of the pargraph",
                        "name": "title",
                        "indexInverted": True,
                        "moduleConfig": {
                            "text2vec-huggingface": {
                                "skip": True,
                                "vectorizePropertyName": True,
                            }
                        }
                    },
                    {
                        "dataType": ["text"],
                        "description": "paragraph content",
                        "name": "content",
                        "indexInverted": True,
                        "moduleConfig": {
                            "text2vec-huggingface": {
                                "skip": True,
                                "vectorizePropertyName": False,
                            }
                        }
                    },
                    {
                        "dataType": ["int"],
                        "description": "Order of the paragraph",
                        "name": "order",
                        "indexInverted": True,
                        "moduleConfig": {
                            "text2vec-huggingface": {
                                "skip": True,
                                "vectorizePropertyName": False,
                            }
                        }
                    },
                ]
            }
    schema_article = {
                "class": "Article",
                "description": "A Coppermind article with a title and references",
                "vectorizer": "none",
                "moduleConfig": {
                    "text2vec-huggingface": {
                        "model": "sentence-transformers/all-MiniLM-L6-v2",
                        "options": {
                            "waitForModel": True,
                            "useGPU": True,
                            "useCache": True
                            },
                    },
                    "generative-palm":{
                        "projectId": "Generative Language Client",
                        "apiEndpoint": "generativelanguage.googleapis.com",
                        "modelId": "gemini-1.5-flash-latest",
                    },
                },
                "properties": [
                    {
                        "name": "title",
                        "description": "Title of the article",
                        "dataType": ["string"],
                        "indexInverted": True,
                        "moduleConfig":{"text2vec-huggingface":{"skip": True}}
                    },
                    {
                        "name": "hasParagraphs",
                        "description": "List of paragraphs from the article",
                        "dataType": ["Paragraph"],
                    },
                    {
                        "name": "linksToArticles",
                        "description": "Cross-references from the article",
                        "dataType": ["Article"],
                    }
                ]
            }
    for cls in [schema_paragraph,schema_article]:
        client.collections.create_from_dict(cls)
    
    para = client.collections.get("Paragraph")
    para.config.add_reference(
        ReferenceProperty(name="inArticle",
                          target_collection="Article"))
    
    

In [5]:
def show_collection_refs(client, collection_name, ref_name, return_proprties):
    query_col = client.collections.get(collection_name)
    response = query_col.query.fetch_objects(
        return_references=[
            QueryReference(link_on=ref_name, return_properties=return_proprties)])

    for o in response.objects:
        print(o.properties["title"])
        if ref_name in o.references:
            for ref_obj in o.references[ref_name].objects:
                print(ref_obj.properties)
        else: continue

In [6]:
def fresh_start(client):
    client.collections.delete_all()
    create_weaviate_schema(client)

In [66]:
with weaviate.connect_to_local(headers=header) as client:
    if not client.is_connected(): client.connect()
    fresh_start(client)
    tsts = client.collections.get("Paragraph")
    print(tsts)

<weaviate.Collection config={
  "name": "Paragraph",
  "description": "a paragraph with a header and and parent Article",
  "generative_config": null,
  "inverted_index_config": {
    "bm25": {
      "b": 0.75,
      "k1": 1.2
    },
    "cleanup_interval_seconds": 60,
    "index_null_state": false,
    "index_property_length": false,
    "index_timestamps": false,
    "stopwords": {
      "preset": "en",
      "additions": null,
      "removals": null
    }
  },
  "multi_tenancy_config": {
    "enabled": false,
    "auto_tenant_creation": false,
    "auto_tenant_activation": false
  },
  "properties": [
    {
      "name": "title",
      "description": "Title/header of the pargraph",
      "data_type": "text",
      "index_filterable": true,
      "index_searchable": true,
      "nested_properties": null,
      "tokenization": "word",
      "vectorizer_config": {
        "skip": false,
        "vectorize_property_name": false
      },
      "vectorizer": "text2vec-palm"
    },
    {
 

In [2]:
manager = SourceManager()
articles = manager.load_json("articles.jsonl")

In [3]:
page_titles = [
    "Kaladin",
    "Pits of Hathsin",
    "Allomancy",
    "Cosmere",
    "Honor's Perpendicularity",
    "Cephandrius",
    "Hoid"
]
manager._init_mwclient()
data = manager.prep_data_graph(page_titles)
manager.save_json(data)

MWClient Connected with coppermind.net
MWClient unable to connect with coppermind.net
Processing Kaladin
Processing Pits of Hathsin
Processing Allomancy
Processing Cosmere
Processing Honor's Perpendicularity
Processing Cephandrius
Processing Hoid
articles.jsonl saved
articles.jsonl saved


In [92]:
data[-1]

{'title': 'Hoid',
 'links': ['Sons of Honor',
  'Knights Radiant',
  'Awakening',
  'Hallandren',
  'sa2-67',
  'tress-epilogue',
  'The Sunlit Man',
  'Lift',
  'mb3-27',
  'Shattered Plains',
  'Sixth of the Dusk',
  'wob_ref-2248',
  'Shardblade',
  'wob_ref-6162',
  'wob_ref-10773',
  'Diem',
  'Southern Peoples of Scadrial',
  'True Desolation',
  'wob_ref-16524',
  'wob_ref-15354',
  'mb7-part=ars',
  'wob_ref-1753',
  'kandra',
  'Rosharan system',
  'Tears of Edgli',
  'Darkside',
  'Elantris (book)',
  'Bavadin',
  'Allomancy',
  'Wandersail',
  'Shallan Davar',
  'Cognitive Realm',
  "Cultivation's Perpendicularity",
  'Elantrian',
  'Khriss',
  'wob_ref-9218',
  'Tanavast',
  'Yomen-Ostlin wedding dinner',
  '#Known Aliases',
  'wob_ref-2101',
  'Ishikk',
  'Waxillium Ladrian',
  'Mistborn',
  'sa1-epilogue',
  'wob_ref-3759',
  'wob_ref-15271',
  'Vasher',
  'wob_ref-2567',
  'Inquisitor',
  'wob_ref-1523',
  'wob_ref-656',
  'lighteyes',
  'Forger',
  'wob_ref-7135',
  'Cr

In [81]:
from weaviate.util import generate_uuid5


model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
with weaviate.connect_to_local(headers=header) as client:
    if not client.is_connected(): client.connect()
    fresh_start(client) # delete old entries. Start fresh
    
    col_articles = client.collections.get("Article")
    col_paragraphs = client.collections.get("Paragraph")
    # Add without cross refs first
    for art in articles:
        art_properties = {
            "title": art["title"],
        }
        art_uuid = generate_uuid5(art_properties)
        vector = model.encode(art["title"])
        col_articles.data.insert(uuid=art_uuid, properties=art_properties, vector=vector)
        
        # print(col_articles.data.insert(properties=art_properties))
        if art['sections'] is None:
            print(f"No sections: {art['title']}")
        else:
            for para in art['sections']:
                para_properties={
                    "title": para["title"],
                    "content": para["content"],
                    "order": para["order"],
                }
                para_uuid = generate_uuid5(para_properties)
                title_vector = model.encode(para_properties['title'])
                content_vector = model.encode(para_properties['content'])
                col_paragraphs.data.insert(
                    uuid=para_uuid, 
                    properties=para_properties, 
                    vector={
                        'title': title_vector,
                        'content': content_vector
                    })
                # make two way reference
                col_paragraphs.data.reference_add(
                    from_uuid=para_uuid,
                    from_property="inArticle",
                    to=art_uuid
                    )
                col_articles.data.reference_add(
                    from_uuid=art_uuid,
                    from_property="hasParagraphs",
                    to=para_uuid
                    )



No sections: Cephandrius


In [70]:

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
with weaviate.connect_to_local(headers=header) as client:
    if not client.is_connected(): client.connect()
    fresh_start(client) # delete old entries. Start fresh
    
    col_articles = client.collections.get("Article")
    col_paragraphs = client.collections.get("Paragraph")
    # Add without cross refs first
    for art in articles:
        art_properties = {
            "title": art["title"],
        }
        art_uuid = generate_uuid5(art_properties)
        vector = model.encode(art["title"])
        col_articles.data.insert(uuid=art_uuid, properties=art_properties, vector=vector)
            
        # print(col_articles.data.insert(properties=art_properties))
        if art['sections'] is None:
            print(f"No sections: {art['title']}")
        else:
            for para in art['sections']:
                para_properties={
                    "title": para["title"],
                    "content": para["content"],
                    "order": para["order"],
                }
                para_uuid = generate_uuid5(para_properties)
                title_vector = model.encode(para_properties['title'])
                content_vector = model.encode(para_properties['content'])
                col_paragraphs.data.insert(
                    uuid=para_uuid, 
                    properties=para_properties, 
                    vector={
                        'title': title_vector,
                        'content': content_vector
                    })
                # make two way reference
                col_paragraphs.data.reference_add(
                    from_uuid=para_uuid,
                    from_property="inArticle",
                    to=art_uuid
                    )
                col_articles.data.reference_add(
                    from_uuid=art_uuid,
                    from_property="hasParagraphs",
                    to=para_uuid
                    )



UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'text2vec-openai vectorizer module not part of the property'}]}.

In [88]:
from weaviate.classes.query import QueryReference
with weaviate.connect_to_local(headers=header) as client:
    if not client.is_connected(): client.connect()
    # show_collection_refs(client, "Article", "hasParagraphs", "title")
    collection_name = "Article"
    ref_name = "hasParagraphs"
    return_proprties ="title"

Honor's Perpendicularity
{'title': "Honor's Perpendicularity"}
{'title': "Honor's Perpendicularity"}
{'title': 'At Thaylen Field'}
{'title': 'At Thaylen Field'}
{'title': 'At Thaylen Field'}
{'title': 'Trivia'}
Allomancy
{'title': 'Allomancy'}
{'title': 'Mechanics'}
{'title': 'Mechanics'}
{'title': 'Mechanics'}
{'title': 'Mechanics'}
{'title': 'Mechanics'}
{'title': 'Mechanics'}
{'title': 'Gaining Allomantic Abilities'}
{'title': 'Gaining Allomantic Abilities'}
{'title': 'Gaining Allomantic Abilities'}
{'title': 'Snapping'}
{'title': 'Snapping'}
{'title': 'Snapping'}
{'title': 'Snapping'}
{'title': 'Mist Sickness'}
{'title': 'Mist Sickness'}
{'title': 'Mist Sickness'}
{'title': 'Savantism'}
{'title': 'Types of Allomancers'}
{'title': 'Types of Allomancers'}
{'title': 'Twinborn and Compounders'}
{'title': 'Twinborn and Compounders'}
{'title': 'Allomantic Metals'}
{'title': 'Allomantic Metals'}
{'title': 'Allomantic Metals'}
{'title': 'Allomantic Metals'}
{'title': 'God Metals'}
{'title'

In [42]:
with weaviate.connect_to_local(headers=header) as client:
    if not client.is_connected(): client.connect()
    collection = client.collections.get("Paragraph")
    for item in collection.iterator():
        print(item.uuid, item.properties)

In [36]:
client.collections.get("Paragraph").iterator()

In [61]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
for i in range(500):
    embeddings = model.encode(sentences)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [137]:
embeddings

array([[ 6.76569119e-02,  6.34959862e-02,  4.87131625e-02,
         7.93049634e-02,  3.74480709e-02,  2.65275245e-03,
         3.93749885e-02, -7.09843030e-03,  5.93614168e-02,
         3.15370075e-02,  6.00980520e-02, -5.29052801e-02,
         4.06067595e-02, -2.59308498e-02,  2.98428256e-02,
         1.12689065e-03,  7.35148787e-02, -5.03818244e-02,
        -1.22386575e-01,  2.37028543e-02,  2.97265109e-02,
         4.24768552e-02,  2.56337635e-02,  1.99514860e-03,
        -5.69190569e-02, -2.71598138e-02, -3.29035595e-02,
         6.60249069e-02,  1.19007170e-01, -4.58791293e-02,
        -7.26214573e-02, -3.25840563e-02,  5.23413755e-02,
         4.50553223e-02,  8.25305190e-03,  3.67024280e-02,
        -1.39415143e-02,  6.53918609e-02, -2.64272187e-02,
         2.06402605e-04, -1.36643695e-02, -3.62810344e-02,
        -1.95043758e-02, -2.89738402e-02,  3.94270197e-02,
        -8.84090811e-02,  2.62421113e-03,  1.36713935e-02,
         4.83062640e-02, -3.11566275e-02, -1.17329195e-0